# Urllib

Urllib是Python自带的标准库，通常用于爬虫开发，API数据获取与测试。

Urllib在Python2和Python3中的用法有着较大的不同。

Python2中Urllib中分为urllib和urllib2两个模块，urllib2可以接收一个Request对象，而Urllib只能接收一条URL，这意味着不能伪装User-Agent字符串等。

Python3中就把Python2里的urllib和urllib2合并了。

## 常用接口

- urllib.request.urlopen：urllib最基本的使用功能，用于访问URL（请求链接）的唯一方法。
- urllib.request.Request：声明request对象，该对象可自定义请求头（header）、请求方式等信息。
- urllib.request.ProxyHandler：动态设置代理IP池，可加载请求对象。
- urllib.request.HTTPCookieProcessor：设置Cookies对象，可加载请求对象。
- urllib.request.build_opener（）：创建请求对象，用于代理IP和Cookies对象加载。
- urllib.parse.urlencode（data）.encode（'utf-8'）：请求数据格式转换。
- urllib.parse.quote（url）：URL编码处理，主要对URL上的中文等特殊符号编码处理。
- urllib.parse.unquote（url）：URL解码处理，将URL上的特殊符号还原。

## urlopen

urlopen用于向服务器端发送一个请求，可以是GET或POST方法，这取决于第二个参数`data`字段是否为空。传入的请求可以是一个`url`字符串，也可以是一个`urllib.Request`对象。

它返回一个`http.client.HTTPResponse`对象，这个对象提供了下面的方法：
- read()
- readline()
- readlines()
- fileno()
- close()
- info()：返回HTTPMessage对象，表示远程服务器返回的头信息
- getcode()：返回HTTP状态码
- geturl() 返回请深圳市的URL

In [2]:
import urllib

help(urllib.request.urlopen)

Help on function urlopen in module urllib.request:

urlopen(url, data=None, timeout=<object object at 0x7fc83ffbedd0>, *, cafile=None, capath=None, cadefault=False, context=None)
    Open the URL url, which can be either a string or a Request object.
    
    *data* must be an object specifying additional data to be sent to
    the server, or None if no such data is needed.  See Request for
    details.
    
    urllib.request module uses HTTP/1.1 and includes a "Connection:close"
    header in its HTTP requests.
    
    The optional *timeout* parameter specifies a timeout in seconds for
    blocking operations like the connection attempt (if not specified, the
    global default timeout setting will be used). This only works for HTTP,
    HTTPS and FTP connections.
    
    If *context* is specified, it must be a ssl.SSLContext instance describing
    the various SSL options. See HTTPSConnection for more details.
    
    The optional *cafile* and *capath* parameters specify a set of

下面这段程序由于网站设置了防爬，所以返回了418错误码。

>418 I'm a teapot
   Any attempt to brew coffee with a teapot should result in the error
   code "418 I'm a teapot". The resulting entity body MAY be short and
   stout.

In [ ]:
from urllib import request

with request.urlopen('https://movie.douban.com/', data=None, timeout=2) as response:
    data = response.read()
    print('Status:', response.status, response.reason)
    for k, v in .getheaders():
        print('%s: %s' % (k, v))
    print('DataLen:', len(data.decode('utf-8')))

## 通过Request对象来模拟浏览器访问

In [11]:
from urllib import request
url = 'https://movie.douban.com/'
# 设置请求头部
headers = {
    'User-Agent': 'Mozilla/6.0 (iPhone; CPU iPhone OS 8_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/8.0 Mobile/10A5376e Safari/8536.25',
    'Referer': 'https://movie.douban.com/',
    'Connection': 'keep=alive'
}
# 创建Requset对象
req = request.Request(url=url, headers=headers)
html = open('movie.html', 'wb')
with request.urlopen(req) as response:
    data = response.read()
    print('Status:', response.status, response.reason)
    for k, v in response.getheaders():
        print('%s: %s' % (k, v))
    print('DataLen:', len(data.decode('utf-8')))
    html.write(data)
html.close()

Status: 200 OK
Date: Fri, 20 Mar 2020 01:27:40 GMT
Content-Type: text/html; charset=utf-8
Transfer-Encoding: chunked
Connection: close
Vary: Accept-Encoding
Vary: Accept-Encoding
X-Xss-Protection: 1; mode=block
X-Douban-Mobileapp: 0
Expires: Sun, 1 Jan 2006 01:00:00 GMT
Pragma: no-cache
Cache-Control: must-revalidate, no-cache, private
Set-Cookie: ll="118282"; path=/; domain=.douban.com; expires=Sat, 20-Mar-2021 01:27:40 GMT
Set-Cookie: bid=XWHC-mwW6OM; Expires=Sat, 20-Mar-21 01:27:40 GMT; Domain=.douban.com; Path=/
X-DOUBAN-NEWBID: XWHC-mwW6OM
X-DAE-App: movie
X-DAE-Instance: default
Server: dae
X-Content-Type-Options: nosniff
DataLen: 66659


## 通过代理来访问Web Server

代理的基本原理就是客户端发送给服务器的请求，不是直接发送给服务端，而是先发送给了代理服务器，然后由代理服务器再向服务器来请求，代理服务器拿到响应后，再发送回客户端。

![](../images/2012120511054068.png)

比较著名的代理软件有[Squid](http://www.squid-cache.org/)，另外Web抓包工具Fiddler本质上就是一个代理服务。

In [ ]:
import urllib.request

url = 'https://movie.douban.com'
# 设置代理IP
proxy_handler = urllib.request.ProxyHandler({
    'http': '218.56.132.157:8080',
    'https': '183.30.197.29:9797'
})
# 使用build_opener()函数来创建带有代理IP功能的opener对象
opener = urllib.request.build_opener(proxy_handler)
response = opener.open(url)
html = response.read().decode('utf-8')
f = open('html.txt', 'w', encoding='utf8')
f.write(html)
f.close()

## 使用Cookies

Cookies用于在浏览器侧保存用户的信息，比如当我们访问某个网站，提交数据实现了用户登录后，会生成带有登录状态的Cookies，这时可以将Cookies保存在本地文件中，下次程序运行的时候，可以直接读取Cookies文件来实现用户登录。

特别是对于一些复杂的登录，如验证码、手机短信验证登录的这类网站，使用Cookies能简单解决重复登录的问题。

In [13]:
import urllib.request
from http import cookiejar

url = 'https://movie.douban.com/'
headers = {
    'User-Agent': 'Mozilla/6.0 (iPhone; CPU iPhone OS 8_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/8.0 Mobile/10A5376e Safari/8536.25',
    'Referer': 'https://movie.douban.com/',
    'Connection': 'keep=alive'
}
req = request.Request(url=url, headers=headers)

filename='cookie.txt'
cookie = cookiejar.MozillaCookieJar(filename)
# 创建HTTPCookieProcessor cookie处理器
handler = urllib.request.HTTPCookieProcessor(cookie)
# 创建自定义opener
opener = urllib.request.build_opener(handler)
response = opener.open(req)
cookie.save()

In [14]:
# 读取cookie
filename='cookie.txt'
cookie = cookiejar.MozillaCookieJar()
cookie.load(filename)
print(cookie)

<MozillaCookieJar[<Cookie bid=5N4DgDZzu7U for .douban.com/>, <Cookie ll="118282" for .douban.com/>]>


## 证书验证

安全性较好的网络，一般都通过SSL协议对整个数据传输过程进行加密，这样做可以防止有第三方来假冒网络，窃取用户的核心信息。

所以一般网站都会提供一个证书来证明自己的身份。

反过来服务器有时候也需要验证用户的身份，那么就需要用户的证书。

我们可以在进行服务器访问时，可以选择关闭对远端的证书校验。

In [16]:
import urllib.request
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
url = 'https://kyfw.12306.cn/otn/leftTicket/init'
response = urllib.request.urlopen(url)
print(response.getcode())

200


## POST请求

`urllib.request.ulropen()`方法是根据参数`data`字段是否为空，来判断请求是POST还是GET的。

In [17]:
import urllib.request
import urllib.parse

url = 'https://movie.douban.com/'
headers = {
    'User-Agent': 'Mozilla/6.0 (iPhone; CPU iPhone OS 8_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/8.0 Mobile/10A5376e Safari/8536.25',
    'Referer': 'https://movie.douban.com/',
    'Connection': 'keep=alive'
}
data = {
    'value': 'true'
}
data = urllib.parse.urlencode(data).encode('utf-8')
req = request.Request(url=url, headers=headers, data=data)
response = urllib.request.urlopen(req)
print(response.getcode())

200


请求参数中有中文，那么需要对中文进行编码

In [18]:
str = '苹果'
print(urllib.parse.quote(str))

%E8%8B%B9%E6%9E%9C
